アルゴリズムその２

## セクターが違う5つの銘柄を毎週ポジションを調整しながら保有し続ける

- 調整の方法は、各銘柄の３０日平均と１０日平均のDiffをとって、それが３０日平均からみてどのくらいの大きさなのかを確認
- その大きさを元に自己資金に対して何パーセント投資するかを決める



## 終値の10日平均と30日平均の差を算出する

In [ ]:
df = get_pricing(["AAPL"], start_date="2016-1-1", end_date="2017-11-20", fields="close_price", frequency='daily',)
#defaultはコラム名が銘柄オブジェクトなのでアクセスしづらいため、銘柄文字列に変更
df.columns = map(lambda x: x.symbol, df.columns)

In [ ]:
df.tail()

In [ ]:
df["MA30"] = df["AAPL"].rolling(30).mean()
df["MA10"] = df["AAPL"].rolling(10).mean()

In [ ]:
df.tail()

In [ ]:
df.plot()

In [ ]:
df["Weight"] = (df["MA10"] - df["MA30"]) / df["MA30"] 

In [ ]:
df.tail()

In [ ]:
df.plot(secondary_y ="Weight")

In [ ]:
# 5銘柄の過去データを取得
hist = get_pricing(["FB", "JPM", "JNJ", "DIS", "KO"], start_date="2016-1-1", end_date="2017-11-20", fields="price")

In [ ]:
len(hist)


In [ ]:
hist.tail()

In [ ]:
## 30日MAと10日MAを出力
hist30MA=hist.rolling(30).mean()
hist10MA=hist.rolling(10).mean()

In [ ]:
hist10MA.tail()

In [ ]:
raw_weights = (hist10MA - hist30MA) / hist30MA

In [ ]:
raw_weights.tail()


In [ ]:
raw_weights.loc["2017-11-20"]

In [ ]:
raw_weights.ix["2017-11-20"].abs()

In [ ]:
x = raw_weights.ix["2017-11-20"].abs().sum()
x

In [ ]:
normalized_weights = raw_weights.ix["2017-11-20"] / x
normalized_weights

In [ ]:
normalized_weights[normalized_weights > 0].index.tolist()

In [ ]:
normalized_weights[normalized_weights < 0].index.tolist()

In [ ]:
normalized_weights.index.tolist()

In [ ]:
for security in normalized_weights.index:
    print (security, normalized_weights[security])

---
アルゴリズムその３
## Pipelineを使おう


アルゴリズムその２では、最初から５つの銘柄を決めて取引しました．

その３では、アメリカの株式市場で取引されているほとんどすべての銘柄の中から，
条件にあった銘柄を毎週探しだし取引することにしましょう．

つまり

1. 毎週月曜日の朝一に
1. アメリカの株式市場で取引されている全銘柄から，SP500に入っている銘柄だけを取得し
1. 過去３０日と１０日の平均を出して、
1. Diffの絶対値の大きい順に取得し
1. その２と同様の取引を行う。
1. ただし、金曜の取引終了時間に全部クローズする

これを瞬時に行うためのAPIがQuantopianには用意されています

## Pipelineを初期化


In [ ]:
from quantopian.pipeline import Pipeline
from quantopian.research import run_pipeline
# 空のパイプラインを作る
pipe = Pipeline()  


In [ ]:
# pipeline オブジェクトを実行．
results = run_pipeline(pipe, start_date="2017-11-1", end_date="2017-11-20")

In [ ]:
#results
len(results.ix["2017-11-1"])

### ほしい情報を追加,要らない情報を削除
パイプラインにほしい情報を追加していきます．
情報は，株価や出来高だけじゃなく，株価の平均やDiffなど演算した結果や，その銘柄がどの業種の銘柄なのか，等色々な情報が取得出来ます．
取得した情報を元に，必要な情報だけをFilterして，要らない情報を削除することもできます．

どんなデータがあるのかは https://www.quantopian.com/data で確認できます．



In [ ]:
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.factors import SimpleMovingAverage
from quantopian.pipeline.filters.morningstar import Q500US

# パイプラインを初期化
pipe = Pipeline()

# USEquityPricingからcloseデータを取得し、 'close' という名前=コラム名をつける
pipe.add(USEquityPricing.close.latest, 'close') 

# SP500銘柄だけに絞る
pipe.set_screen(Q500US())

## 各銘柄，過去10日間の平均を出す
pipe.add(SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=10), 'SMA10') 
## 同様に30日間の移動平均
pipe.add(SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=30), 'SMA30') 




In [ ]:
results = run_pipeline(pipe, start_date="2017-11-1", end_date="2017-11-20")


In [ ]:
results.ix["2017-11-1"]

In [ ]:

pipe = Pipeline()
pipe.add(USEquityPricing.close.latest, 'close') 
pipe.set_screen(Q500US())

SMA10 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=10)
SMA30 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=30)

pipe.add(SMA10, 'SMA10') 
pipe.add(SMA30, 'SMA30') 

## raw_weights 
raw_weights = (SMA10 - SMA30) / SMA30
abs_raw_weights = raw_weights.abs()

pipe.add(raw_weights, 'raw_weights')
pipe.add(abs_raw_weights, 'abs_raw_weights')

results = run_pipeline(pipe, start_date="2017-11-1", end_date="2017-11-15")


In [ ]:
## 2017-11-1 の結果だけ取得
output = results.ix["2017-11-1"]
output

In [ ]:
security_list = output.sort_values(by = "abs_raw_weights", ascending=False).head() 

security_list

In [ ]:
security_list["normalize_weights"] = security_list["raw_weights"] / security_list["abs_raw_weights"].sum()

In [ ]:
security_list